In [82]:
import pandas as pd
import cohere
import time
from annoy import AnnoyIndex
co = cohere.Client('bE6Is3wvtmXyHtgnCQocDIgdH7PcYwdR21ZhnXgN') 

def embeddings(texts,sleep_time=5):
    # add a wait time to simulate a long running process
    time.sleep(sleep_time)
    response = co.embed(
        model='large',
        texts=list(texts), 
        truncate='LEFT').embeddings
    return response

df = pd.read_excel('coherefulllistoflinks.xlsx')


df['embeddings'] = embeddings(df['text'])
# drop rows frm text_df that havve less than 8 words
df = df[df['text'].str.split().str.len() > 10]
# Create the search index, pass the size of embedding
search_index = AnnoyIndex(4096, 'angular')
# Add all the vectors to the search index, these are stored in the dataframe 'post_members['embeddings']'
for i, vector in enumerate(df['embeddings']):
    search_index.add_item(i, vector)
# Build the search index
search_index.build(10)
#save the search index
search_index.save('search_index2.ann')



True

In [83]:
def search(query, n_results, df, search_index, co):
    # Get the query's embedding
    query_embed = co.embed(texts=[query],
                    model="large",
                    truncate="LEFT").embeddings

    # Get the nearest neighbors and similarity score for the query and the embeddings, append it to the dataframe
    nearest_neighbors = search_index.get_nns_by_vector(query_embed[0], n_results, include_distances=True)
    # filter the dataframe to only include the nearest neighbors using the index
    df = df[df.index.isin(nearest_neighbors[0])]
    df['similarity'] = nearest_neighbors[1]
    df['nearest_neighbors'] = nearest_neighbors[0]
    df = df.sort_values(by='nearest_neighbors', ascending=False)
    return df

question = "How can I build cohere AI into my chatbot?"
# run function to search for the query"'What are usage examples for the generate endpoint?'"
test = search(question, 5, df, search_index, co)
test

C:\Users\Elle\AppData\Local\Temp\ipykernel_14488\1850723036.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['similarity'] = nearest_neighbors[1]
C:\Users\Elle\AppData\Local\Temp\ipykernel_14488\1850723036.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nearest_neighbors'] = nearest_neighbors[0]


,text,link,title,Category,Type,embeddings,similarity,nearest_neighbors
287,Use Cohere’s Classify for intent recognition a...,https://cohere.ai/use-case-intent-recognition,Intent Recognition,Use Cases,Product Documentation,"[0.83154297, -1.0810547, 0.84228516, -0.514160...",0.927897,288
40,Our vision for chatbots is a chatbot that trul...,https://www.youtube.com/watch?v=RW3xIDRoFec,How Startups Can Use NLP to Build a Competitiv...,Cohere Talks,Video,"[-0.12194824, -0.20422363, 3.0195312, -2.47851...",0.883681,287
288,"With Cohere’s Classify, your AI-powered chatbo...",https://cohere.ai/use-case-intent-recognition,Intent Recognition,Use Cases,Product Documentation,"[1.1865234, -0.6411133, 0.50341797, -0.0636596...",0.936381,284
179,"[""Cohere's API is created to help you build na...",https://docs.cohere.ai/docs/quick-start-guides,Quickstart Tutorials,Guides and Concepts,User Documentation,"[0.14013672, -1.1650391, 1.3310547, 0.01342010...",0.895938,179
284,Keywords aren’t enough. Many chatbot platforms...,https://cohere.ai/use-case-intent-recognition,Intent Recognition,Use Cases,Product Documentation,"[0.49438477, -0.22241211, 0.53564453, -1.51562...",0.916464,40


In [84]:
def gen_answer(q, para): 
    response = co.generate( 
        model='command-xlarge-20221108', 
        prompt=f'Paragraph:{para}\n\nAnswer the question using this paragraph.\n\nQuestion: {q}\nAnswer:', 
        max_tokens=100, 
        temperature=0.4, 
        k=0, 
        p=0.75, 
        frequency_penalty=0, 
        presence_penalty=0, 
        stop_sequences=[], 
        return_likelihoods='NONE')
        #num_generations=5) 
    return response.generations[0].text

# for each row in the dataframe, generate an answer
test['answer'] = test.apply(lambda x: gen_answer(question, x['text']), axis=1)
test



,text,link,title,Category,Type,embeddings,similarity,nearest_neighbors,answer
287,Use Cohere’s Classify for intent recognition a...,https://cohere.ai/use-case-intent-recognition,Intent Recognition,Use Cases,Product Documentation,"[0.83154297, -1.0810547, 0.84228516, -0.514160...",0.927897,288,\n\nDevelopers can further finetune Cohere's l...
40,Our vision for chatbots is a chatbot that trul...,https://www.youtube.com/watch?v=RW3xIDRoFec,How Startups Can Use NLP to Build a Competitiv...,Cohere Talks,Video,"[-0.12194824, -0.20422363, 3.0195312, -2.47851...",0.883681,287,\n\nOur vision for chatbots is a chatbot that ...
288,"With Cohere’s Classify, your AI-powered chatbo...",https://cohere.ai/use-case-intent-recognition,Intent Recognition,Use Cases,Product Documentation,"[1.1865234, -0.6411133, 0.50341797, -0.0636596...",0.936381,284,"\n\nWith Cohere's Classify, your AI-powered ch..."
179,"[""Cohere's API is created to help you build na...",https://docs.cohere.ai/docs/quick-start-guides,Quickstart Tutorials,Guides and Concepts,User Documentation,"[0.14013672, -1.1650391, 1.3310547, 0.01342010...",0.895938,179,\n\nCohere's AI is created to help you build n...
284,Keywords aren’t enough. Many chatbot platforms...,https://cohere.ai/use-case-intent-recognition,Intent Recognition,Use Cases,Product Documentation,"[0.49438477, -0.22241211, 0.53564453, -1.51562...",0.916464,40,\n\nMany chatbot platforms are powered by basi...


In [85]:
# create a variable that contains each answer
answers = test['answer'].tolist()
answers

["\n\nDevelopers can further finetune Cohere's large language models with their own dataset per their business or industry. Set the road rules, define your core customer intent categories, and train Cohere's Classify model by providing it with example customer queries for each category. Start the engine, Classify then combines this custom understanding of your intent categories with a deep understanding of language and context provided by our large language models to create a finetuned model, ready to handle your customer data.",
 "\n\nOur vision for chatbots is a chatbot that truly understands your meaning and is able to carry the conversation in a way that seems very natural. I'm going to show you an example very soon.",
 "\n\nWith Cohere's Classify, your AI-powered chatbots or conversational tools can consistently and accurately detect the intent behind user requests — including compound queries with multiple data points to consider. As a result, your bots are more intelligent and y

In [86]:
def gen_better_answer(ques, ans): 
    response = co.generate( 
        model='command-xlarge-20221108', 
        prompt=f'Answers:{ans}\n\nQuestion: {ques}\n\nGenerate a new answer that uses all the answers and provides additonal information.', 
        max_tokens=100, 
        temperature=0.4, 
        k=0, 
        p=0.75, 
        frequency_penalty=0, 
        presence_penalty=0, 
        stop_sequences=[], 
        return_likelihoods='NONE')
        #num_generations=5) 
    return response.generations[0].text

# run the function to generate a better answer
answ = gen_better_answer(question, answers)
answ

'\n\n"With Cohere\'s Classify, your AI-powered chatbots or conversational tools can consistently and accurately detect the intent behind user requests — including compound queries with multiple data points to consider. As a result, your bots are more intelligent and your customers are more satisfied with their interaction with your business."'

In [87]:
# for each row print the question, paragraph, and answer
for index, row in test.iterrows():
    print(question)
    print('Answer: ', row['answer'])
    print('Similarity: ', row['similarity'])
    print('Paragraph: ', row['text'])

    print('-----------------------------')

How can I build cohere AI into my chatbot?
Answer:  

Developers can further finetune Cohere's large language models with their own dataset per their business or industry. Set the road rules, define your core customer intent categories, and train Cohere's Classify model by providing it with example customer queries for each category. Start the engine, Classify then combines this custom understanding of your intent categories with a deep understanding of language and context provided by our large language models to create a finetuned model, ready to handle your customer data.
Similarity:  0.9278966188430786
Paragraph:  Use Cohere’s Classify for intent recognition and text classification. Starting from the same Transformer architecture as Google’s Search and Translate, Cohere’s Classify endpoint can read, understand, and label the intent behind unstructured customer queries with exceptional speed and accuracy. Developers can further finetune Cohere’s large language models with their own 